In [2]:
import sys
#sys.path.append('/global/common/cori_cle7/software/jupyter/cori/20-09/lib/python3.8/site-packages')

import yt
import numpy as np
from yt import derived_field
import matplotlib.pyplot as plt
import matplotlib.axes as ax
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib import pylab
from yt.visualization.volume_rendering.transfer_function_helper import TransferFunctionHelper
from yt.visualization.volume_rendering.render_source import VolumeSource
import scipy.fft as fft
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as ticker
from astropy import units as u
from matplotlib import colors
import imageio
import ffmpeg
import glob
import pdf2image

In [ ]:
#3 flavor neutrino derived fields: Number Density
#normalize by the trace
@derived_field(name="Norm", units="dimensionless", sampling_type="cell",force_override=True)
def _Norm(field, data):
    return np.abs(data["N00_Re"]) + np.abs(data["N11_Re"]) + np.abs(data["N22_Re"])
@derived_field(name="N01_Norm", units="dimensionless", sampling_type="cell",force_override=True)
def _N01_Norm(field, data):
    return np.abs(data["N00_Re"]) + np.abs(data["N11_Re"])
@derived_field(name="N02_Norm", units="dimensionless", sampling_type="cell",force_override=True)
def _N02_Norm(field, data):
    return np.abs(data["N00_Re"]) + np.abs(data["N22_Re"])
@derived_field(name="N12_Norm", units="dimensionless", sampling_type="cell",force_override=True)
def _N12_Norm(field, data):
    return np.abs(data["N11_Re"]) + np.abs(data["N22_Re"])

#individual flavor pairing magnitudes, 0=electron, 1=mu, 2=tau
@derived_field(name="N01_Mag", units="dimensionless", sampling_type="cell",force_override=True)
def _N01_Mag(field, data):
    return np.sqrt((data["N01_Im"]/data["N01_Norm"])**2 + (data["N01_Re"]/data["N01_Norm"])**2)
@derived_field(name="N02_Mag", units="dimensionless", sampling_type="cell",force_override=True)
def _N02_Mag(field, data):
    return np.sqrt((data["N02_Im"]/data["N02_Norm"])**2 + (data["N02_Re"]/data["N02_Norm"])**2)
@derived_field(name="N12_Mag", units="dimensionless", sampling_type="cell",force_override=True)
def _N12_Mag(field, data):
    return np.sqrt((data["N12_Im"]/data["N12_Norm"])**2 + (data["N12_Re"]/data["N12_Norm"])**2)

#Diagonal components normalized
@derived_field(name="N00_Mag", units="dimensionless", sampling_type="cell",force_override=True)
def _N00_Mag(field, data):
    return data["N00_Re"]/data["Norm"]
@derived_field(name="N11_Mag", units="dimensionless", sampling_type="cell",force_override=True)
def _N11_Mag(field, data):
    return data["N11_Re"]/data["Norm"]
@derived_field(name="N22_Mag", units="dimensionless", sampling_type="cell",force_override=True)
def _N22_Mag(field, data):
    return data["N22_Re"]/data["Norm"]

#total off-diagonal magnitude
@derived_field(name="OffDiag_Mag", units="dimensionless", sampling_type="cell",force_override=True)
def _Diag_Mag(field, data):
    return np.sqrt((data["N01_Im"]/data["Norm"])**2 + (data["N01_Re"]/data["Norm"])**2 + (data["N02_Im"]/data["Norm"])**2 + (data["N02_Re"]/data["Norm"])**2 + (data["N12_Im"]/data["Norm"])**2 + (data["N12_Re"]/data["Norm"])**2)

#off-diagonal phases in degrees for each off-diagonal component is the arctan(Im/Re)
@derived_field(name="N01_Phase", units="dimensionless", sampling_type="cell",force_override=True)
def _N01_Phase(field, data):
    return np.arctan2(data["N01_Im"],data["N01_Re"])*(180/np.pi)
@derived_field(name="N02_Phase", units="dimensionless", sampling_type="cell",force_override=True)
def _N02_Phase(field, data):
    return np.arctan2(data["N02_Im"],data["N02_Re"])*(180/np.pi)
@derived_field(name="N12_Phase", units="dimensionless", sampling_type="cell",force_override=True)
def _N12_Phase(field, data):
    return np.arctan2(data["N12_Im"],data["N12_Re"])*(180/np.pi)

# Generating off-diagonal magnitude slice plots for each plotfile

In [ ]:
yt.funcs.mylog.setLevel(100)
ts = yt.load("/global/project/projectdirs/m3018/Emu/3D_3flavor_5ns/plt?????")

for ds in ts:
    si = yt.SlicePlot(ds, 'x', 'OffDiag_Mag', origin='native')
    si1 = yt.SlicePlot(ds, 'z', 'OffDiag_Mag', origin='native')
    si.set_zlim('OffDiag_Mag', 1e-6,4e-1)
    si1.set_zlim('OffDiag_Mag', 1e-6,4e-1)
    si.set_log('OffDiag_Mag', False)
    si1.set_log('OffDiag_Mag', False)
    si.save("3D_new/{0}_{1}_yz.png".format(ds, 'Off_Diag_Mag'))
    si1.save("3D_new/{0}_{1}_xy.png".format(ds, 'Off_Diag_Mag'))

# Making a Volume Rendering

In [ ]:
sc = yt.create_scene(ds, 'OffDiag_Mag')

source = sc[0]
source.set_field('OffDiag_Mag')
v_min = 1e-6
v_max = 4e-1
tf = yt.ColorTransferFunction((np.log10(v_min),np.log10(v_max)))
tf.add_layers(6, alpha=np.logspace(-1,2,6), colormap = 'arbre')
source.tfh.tf = tf
source.tfh.set_bounds([v_min,v_max])
source.tfh.set_log(True)
source.tfh.grey_opacity = False

sc.camera.focus = ds.domain_center
sc.camera.resolution = 1024
sc.camera.north_vector = [0, 0, 1]
sc.annotate_domain(ds, color=[1, 1, 1, 0.002])

source.tfh.plot('transfer_function.png')

text_string = "T = {:.4f} ns".format(float(ds.current_time.to('ns')))

sc.save_annotated('rendering.png', sigma_clip=4, label_fmt="%.0E", text_annotate=[[(.1, 0.95), text_string]])

In [ ]:
#alternate volume renderings with color determined by the off-diagonal phase
ds = yt.load("/global/project/projectdirs/m3018/Emu/3d_test2_3flavor/plt00050")
sc = yt.create_scene(ds, 'N01_Phase')

source = sc[0]
source.set_field('N01_Phase')
v_min = -180
v_max = 180
#Apparently need to set_log to false before doing anything else
source.tfh.set_log(False)
#apparently you also need this command to actually turn off the log setting!
source.set_log(False)
source.tfh.set_bounds([v_min,v_max])
source.tfh.grey_opacity = False
tf = yt.ColorTransferFunction((v_min,v_max))
tf.add_layers(6, w=2*(v_max - v_min)/6, alpha=[1,1,1,1,1,1], colormap = 'twilight')
source.tfh.tf = tf

sc.camera.focus = ds.domain_center
sc.camera.resolution = 1024
sc.camera.north_vector = [0, 0, 1]
sc.annotate_domain(ds, color=[1, 1, 1, 0.002])

source.tfh.plot('transfer_function.png')

text_string = "T = {:.4f} ns".format(float(ds.current_time.to('ns')))

sc.save_annotated('rendering.png', sigma_clip=4, label_fmt="%.0E", text_annotate=[[(.1, 0.95), text_string]])

# Loop through to apply volume rendering code to each plotfile

In [ ]:
from yt.visualization.volume_rendering.transfer_function_helper import TransferFunctionHelper
from yt.visualization.volume_rendering.api import Scene, VolumeSource

#process all plotfiles
ts = yt.load("/global/project/projectdirs/m3018/Emu/3D_3flavor_5ns/plt?????")
    
for ds in ts:
    sc = yt.create_scene(ds, 'OffDiag_Mag')

    source = sc[0]
    source.set_field('OffDiag_Mag')
    v_min = 1e-6
    v_max = 4e-1
    tf = yt.ColorTransferFunction((np.log10(v_min),np.log10(v_max)))
    tf.add_layers(10, w=0.1*(np.log10(v_max) - np.log10(v_min))/10, alpha=np.logspace(-1,0,10), colormap = 'arbre')
    source.tfh.tf = tf
    source.tfh.set_bounds([v_min,v_max])
    source.tfh.set_log(True)
    source.tfh.grey_opacity = False

    sc.camera.focus = ds.domain_center
    sc.camera.resolution = 1024
    sc.camera.north_vector = [0, 0, 1]
    sc.annotate_domain(ds, color=[1, 1, 1, 0.002])

    source.tfh.plot('3D_new/{0}_tf.png'.format(ds))

    text_string = "T = {:.4f} ns".format(float(ds.current_time.to('ns')))
    
    sc.render()
    
    sc.save_annotated("3D_new/{0}_Off_Diag_Mag_Volume.png".format(ds), sigma_clip=4, label_fmt="%.0E", text_annotate=[[(.1, 0.95), text_string]])

# Subplots

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colorbar

plt.rc('axes', labelsize=18, linewidth=2)
plt.rc('xtick',labelsize=18)
plt.rc('ytick',labelsize=18)
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['ytick.minor.width'] = 2
plt.rcParams["xtick.major.size"] = 6
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["ytick.major.size"] = 6
plt.rcParams["ytick.minor.size"] = 3

rc = {"font.family" : "serif", 
      "mathtext.fontset" : "stix"}
plt.rcParams.update(rc)
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]

yt.funcs.mylog.setLevel(100)
ts = yt.load("/global/cscratch1/sd/srichers/3D_3flavor_5ns_v2/plt?????")

for ds in ts:
    #if int(str(ds)[4]) == 3:
    slc = ds.slice('x', 0.)
    #use fixed resolution buffer to extract 2D slice data object information
    frb = slc.to_frb((64, 'cm'), [256, 256])

    #list of fields to plot ordered by row and then column
    fields = [['N00_Mag','N01_Mag','N02_Mag'],['N01_Phase','N11_Mag','N12_Mag'],['N02_Phase','N12_Phase','N22_Mag']]
    labels = [[r'$N_{ee}$',r'$N_{e\mu}$',r'$N_{e\tau}$'],[r'$\phi_{e\mu}$',r'$N_{\mu\mu}$',r'$N_{\mu\tau}$'],[r'$\phi_{e\tau}$',r'$\phi_{\mu\tau}$',r'$N_{\tau\tau}$']]

    # Set up a 3x3 figure 
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15,15), sharex=True, sharey=True)

    #defining the two colorbars to be used
    cax1 = fig.add_axes([.93, 0.122, 0.03, 0.76])
    norm1 = colors.Normalize(vmin=0, vmax=0.6)
    cmap = plt.get_cmap('viridis')
    cb1 = colorbar.ColorbarBase(cax1, cmap=cmap,norm=norm1,orientation='vertical')
    cb1.set_label(r'$N_{ij} / \mathrm{Tr}(N)$', fontsize=22)
    cb1.ax.tick_params(axis='y', which='both', direction='in')
    cb1.minorticks_on()
    cax1.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
    cax1.yaxis.set_minor_locator(ticker.MultipleLocator(0.02))

    cax2 = fig.add_axes([0.04, 0.122, 0.03, 0.76])
    norm2 = colors.Normalize(vmin=-180, vmax=180)
    cmap2 = plt.get_cmap('twilight')
    cb2 = colorbar.ColorbarBase(cax2, cmap=cmap2,norm=norm2,orientation='vertical')
    cax2.yaxis.set_ticks_position('left')
    cax2.yaxis.set_label_position('left')
    cb2.set_label(r'$\phi_{ij}$ (degrees)', fontsize=22)
    cb2.ax.tick_params(axis='y', which='both', direction='in')
    cax2.minorticks_on()
    cax2.yaxis.set_major_locator(ticker.MultipleLocator(45))
    cax2.yaxis.set_minor_locator(ticker.MultipleLocator(5))

    for i in range(0,3):
        for j in range(0,3):
            ax = axes[i,j]
            ax.minorticks_on()
            ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
            ax.yaxis.set_minor_locator(ticker.MultipleLocator(2))
            ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
            ax.xaxis.set_minor_locator(ticker.MultipleLocator(2))
            ax.tick_params(axis='both', which='both', direction='in', right=True,top=True)
            ax.text(8,58, "{}".format(labels[i][j]), fontsize=18, ha="center", va="center",bbox=dict(facecolor='white', alpha=0.75, edgecolor='.75'))
            #setting the phase plots
            if ax == axes[1,0] or ax == axes[2,0] or ax == axes[2,1]:
                im = ax.imshow(frb[fields[i][j]].d,origin='lower',extent=[0,64,0,64], vmin=-180, vmax=180, cmap='twilight')
                #setting colorbar limits
                im.set_clim(-180,180)
            #setting the N magnitudes plots
            else:
                #origin=lower keeps image from getting flipped, extent gives the grid coordinates in physical units
                im = ax.imshow(frb[fields[i][j]].d,origin='lower',extent=[0,64,0,64], vmin=0., vmax=0.6, cmap='viridis')
                #setting colorbar limits
                im.set_clim(0.,0.6) 

    for a in axes[-1,:]: a.set_xlabel('y (cm)')
    for a in axes[:,0]: a.set_ylabel('z (cm)')

    #plt.tick_params(axis='both', which='both', direction='in', right=True,top=True)
    #increases the spacing between subplots so axis labels don't overlap
    plt.subplots_adjust(hspace=0.05,wspace=0.)
    #fig.tight_layout()
    #plt.show()
    fig.savefig("Emu_slice_phase/{0}_3x3subplot.pdf".format(ds),dpi=300, bbox_inches="tight")
    fig.savefig("Emu_slice_phase/{0}_3x3subplot.png".format(ds),dpi=300, bbox_inches="tight")
    plt.close()

# Code for converting pdf to png

In [2]:
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
import os

pdf_dir = r"/global/u2/n/nmford20/Emu_slice_phase"
os.chdir(pdf_dir)

for pdf_file in glob.glob(os.path.join(pdf_dir, "*.pdf")):
    pages = convert_from_path(pdf_file, 150)
    for page in pages:
        page.save(pdf_file[:-4] +".png", 'png')

# Stitching together images

In [3]:
writer = imageio.get_writer('Emu_slice_phase/3x3_subplots.mp4', fps=10)
for file in sorted(glob.glob("Emu_slice_phase/*3x3subplot.png")):
    writer.append_data(imageio.imread(file))
writer.close()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2390, 1837) to (2400, 1840) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
